<a href="https://colab.research.google.com/github/andersonfurtado/AI4WEBDEV/blob/main/MovieLens_SQLAlchemy_Database_Creation.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install sqlalchemy pandas openpyxl

# Import the libraries
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import urllib.request
import zipfile
import os

# Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Full path to the files
movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# Define the SQLAlchemy models
Base = declarative_base()

class Movie(Base):
    __tablename__ = 'movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'
    userId = Column(Integer, primary_key=True)
    movieId = Column(Integer, primary_key=True)
    rating = Column(Float)
    timestamp = Column(Integer, primary_key=True)

# Create the SQLite database and tables
engine = create_engine('sqlite:///movielens.db')
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the movie table
movies_data = movies.to_dict(orient='records')
for data in movies_data:
    movie = Movie(**data)
    session.add(movie)

# Enter data in the evaluation table
ratings_data = ratings.to_dict(orient='records')
for data in ratings_data:
    rating = Rating(**data)
    session.add(rating)

# Commit session
session.commit()

# Function to get movie recommendations from the database
def get_movie_recommendations_from_db(movie_title, num_recommendations=5):
    movie = session.query(Movie).filter_by(title=movie_title).first()
    if not movie:
        return []

    movie_id = movie.movieId
    users_who_rated_movie = session.query(Rating.userId).filter_by(movieId=movie_id).distinct().all()
    users_who_rated_movie = [u[0] for u in users_who_rated_movie]

    other_movie_ids = session.query(Rating.movieId).filter(Rating.userId.in_(users_who_rated_movie)).distinct().all()
    other_movie_ids = [m[0] for m in other_movie_ids]

    predicted_ratings = []
    for movie_id in other_movie_ids:
        predicted_rating = algo.predict(uid=0, iid=movie_id).est
        predicted_ratings.append((movie_id, predicted_rating))

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movies = session.query(Movie.title).filter(Movie.movieId.in_(top_n_movies)).all()
    return [m[0] for m in recommended_movies]

# Example of use
print(get_movie_recommendations_from_db('Toy Story (1995)'))
